In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
ksi_df = pd.read_csv('Data/KSI.csv')
hazardous_df = pd.read_csv('Data/Hazardous_Driving_Areas_CAN.csv')
ksi_collapse = pd.read_csv('Data/KSI_Collapse_Count.csv', index_col = 0)

In [21]:
pd.set_option('display.max_columns', None)

In [10]:
ksi_df.head(1)

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId
0,-8.844611e+06,5.412414e+06,3387730,892658,2006,2006/03/11 05:00:00+00,852,8,BLOOR ST W,DUNDAS ST W,<Null>,Major Arterial,Toronto and East York,4,11,43.656345,-79.45249,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,unknown,None,<Null>,South,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88,High Park North (88),1


In [12]:
hazardous_df.head(1)

,Geohash,GeohashBounds,Latitude_SW,Longitude_SW,Latitude_NE,Longitude_NE,Location,Latitude,Longitude,City,County,State,Country,ISO_3166_2,SeverityScore,IncidentsTotal,UpdateDate,Version
0,f25eun8,"POLYGON((-73.65234375 45.6948852539063, -73.65...",45.695,-73.652,45.696,-73.651,POINT(-73.65123 45.6955),45.696,-73.651,NaN,NaN,NaN,Canada,CA,0.005,7,2022-02-02,5.1


In [23]:
ksi_collapse = round(ksi_collapse, 3)

In [24]:
hazardous_merge = hazardous_df.merge(ksi_collapse, left_on = ['Latitude', 'Longitude'], right_on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

In [25]:
hazardous_merge

,Geohash,GeohashBounds,Latitude_SW,Longitude_SW,Latitude_NE,Longitude_NE,Location,Latitude,Longitude,City,County,State,Country,ISO_3166_2,SeverityScore,IncidentsTotal,UpdateDate,Version,LATITUDE,LONGITUDE,COUNT
0,dpz97d3,"POLYGON((-79.2979431152344 43.8258361816406, -...",43.826,-79.298,43.827,-79.297,POINT(-79.29696 43.82606),43.826,-79.297,Markham,NaN,Ontario,Canada,CA-ON,0.004,3,2022-02-02,5.1,43.826,-79.297,4
1,dpz968s,"POLYGON((-79.3377685546875 43.8162231445313, -...",43.816,-79.338,43.818,-79.336,POINT(-79.33686 43.81674),43.817,-79.337,Markham,NaN,Ontario,Canada,CA-ON,0.010,5,2022-02-02,5.1,43.817,-79.337,1
2,dpz91zq,"POLYGON((-79.3679809570312 43.8093566894531, -...",43.809,-79.368,43.811,-79.367,POINT(-79.36703 43.80983),43.810,-79.367,Markham,NaN,Ontario,Canada,CA-ON,0.007,3,2022-02-02,5.1,43.810,-79.367,1
3,dpz91zq,"POLYGON((-79.3679809570312 43.8093566894531, -...",43.809,-79.368,43.811,-79.367,POINT(-79.36703 43.80983),43.810,-79.367,Markham,NaN,Ontario,Canada,CA-ON,0.007,3,2022-02-02,5.1,43.810,-79.367,1
4,dpz91zq,"POLYGON((-79.3679809570312 43.8093566894531, -...",43.809,-79.368,43.811,-79.367,POINT(-79.36703 43.80983),43.810,-79.367,Markham,NaN,Ontario,Canada,CA-ON,0.007,3,2022-02-02,5.1,43.810,-79.367,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,dpz3n3j,"POLYGON((-79.5231628417969 43.7750244140625, -...",43.775,-79.523,43.776,-79.522,POINT(-79.52222 43.77532),43.775,-79.522,Vaughan,NaN,Ontario,Canada,CA-ON,0.019,40,2022-02-02,5.1,43.775,-79.522,1
1545,dpz3n3j,"POLYGON((-79.5231628417969 43.7750244140625, -...",43.775,-79.523,43.776,-79.522,POINT(-79.52222 43.77532),43.775,-79.522,Vaughan,NaN,Ontario,Canada,CA-ON,0.019,40,2022-02-02,5.1,43.775,-79.522,2
1546,dpz3n3j,"POLYGON((-79.5231628417969 43.7750244140625, -...",43.775,-79.523,43.776,-79.522,POINT(-79.52222 43.77532),43.775,-79.522,Vaughan,NaN,Ontario,Canada,CA-ON,0.019,40,2022-02-02,5.1,43.775,-79.522,1
1547,dpz3n2b,"POLYGON((-79.530029296875 43.7736511230469, -7...",43.774,-79.530,43.775,-79.529,POINT(-79.52948 43.77386),43.774,-79.529,Vaughan,NaN,Ontario,Canada,CA-ON,0.019,19,2022-02-02,5.1,43.774,-79.529,1


In [9]:
hazardous_merge.sort_values(by = 'SeverityScore', ascending = False)

,Geohash,GeohashBounds,Latitude_SW,Longitude_SW,Latitude_NE,Longitude_NE,Location,Latitude,Longitude,City,County,State,Country,ISO_3166_2,SeverityScore,IncidentsTotal,UpdateDate,Version,LATITUDE,LONGITUDE,COUNT
428,dpz881k,"POLYGON((-79.4476318359375 43.6885070800781, -...",43.689,-79.448,43.690,-79.446,POINT(-79.44747 43.68892),43.689,-79.447,Toronto,NaN,Ontario,Canada,CA-ON,0.257,134,2022-02-02,5.1,43.689,-79.447,1
427,dpz881k,"POLYGON((-79.4476318359375 43.6885070800781, -...",43.689,-79.448,43.690,-79.446,POINT(-79.44747 43.68892),43.689,-79.447,Toronto,NaN,Ontario,Canada,CA-ON,0.257,134,2022-02-02,5.1,43.689,-79.447,1
866,dpz2yrr,"POLYGON((-79.5204162597656 43.7654113769531, -...",43.765,-79.520,43.767,-79.519,POINT(-79.5197 43.76592),43.766,-79.520,Toronto,NaN,Ontario,Canada,CA-ON,0.237,83,2022-02-02,5.1,43.766,-79.520,1
108,dpz8uns,"POLYGON((-79.2718505859375 43.7612915039062, -...",43.761,-79.272,43.763,-79.270,POINT(-79.27157 43.76173),43.762,-79.272,Toronto,NaN,Ontario,Canada,CA-ON,0.231,31,2022-02-02,5.1,43.762,-79.272,1
553,dpz2tg7,"POLYGON((-79.5478820800781 43.6994934082031, -...",43.699,-79.548,43.701,-79.547,POINT(-79.54776 43.69958),43.700,-79.548,Toronto,NaN,Ontario,Canada,CA-ON,0.227,114,2022-02-02,5.1,43.700,-79.548,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,dpz820h,"POLYGON((-79.4476318359375 43.6376953125, -79....",43.638,-79.448,43.639,-79.446,POINT(-79.44704 43.63808),43.638,-79.447,Toronto,NaN,Ontario,Canada,CA-ON,0.001,7,2022-02-02,5.1,43.638,-79.447,1
1265,dpz820h,"POLYGON((-79.4476318359375 43.6376953125, -79....",43.638,-79.448,43.639,-79.446,POINT(-79.44704 43.63808),43.638,-79.447,Toronto,NaN,Ontario,Canada,CA-ON,0.001,7,2022-02-02,5.1,43.638,-79.447,1
325,dpz8660,"POLYGON((-79.354248046875 43.648681640625, -79...",43.649,-79.354,43.650,-79.353,POINT(-79.35345 43.649),43.649,-79.353,Toronto,NaN,Ontario,Canada,CA-ON,0.001,3,2022-02-02,5.1,43.649,-79.353,1
172,dpz8d0e,"POLYGON((-79.3611145019531 43.6843872070312, -...",43.684,-79.361,43.686,-79.360,POINT(-79.36099 43.68545),43.685,-79.361,Toronto,NaN,Ontario,Canada,CA-ON,0.001,3,2022-02-02,5.1,43.685,-79.361,1


In [10]:
hazardous_merge.sort_values(by = 'COUNT', ascending = False)

,Geohash,GeohashBounds,Latitude_SW,Longitude_SW,Latitude_NE,Longitude_NE,Location,Latitude,Longitude,City,County,State,Country,ISO_3166_2,SeverityScore,IncidentsTotal,UpdateDate,Version,LATITUDE,LONGITUDE,COUNT
255,dpz90v0,"POLYGON((-79.420166015625 43.7969970703125, -7...",43.797,-79.420,43.798,-79.419,POINT(-79.4197 43.79796),43.798,-79.420,Toronto,NaN,Ontario,Canada,CA-ON,0.041,48,2022-02-02,5.1,43.798,-79.420,6
1245,dpz2tfg,"POLYGON((-79.5478820800781 43.6967468261719, -...",43.697,-79.548,43.698,-79.547,POINT(-79.5468 43.69714),43.697,-79.547,Toronto,NaN,Ontario,Canada,CA-ON,0.014,20,2022-02-02,5.1,43.697,-79.547,6
1369,dpz8252,"POLYGON((-79.453125 43.6555480957031, -79.4517...",43.656,-79.453,43.657,-79.452,POINT(-79.45239 43.65631),43.656,-79.452,Toronto,NaN,Ontario,Canada,CA-ON,0.046,63,2022-02-02,5.1,43.656,-79.452,5
917,dpz86eh,"POLYGON((-79.3377685546875 43.6541748046875, -...",43.654,-79.338,43.656,-79.336,POINT(-79.33731 43.65498),43.655,-79.337,Toronto,NaN,Ontario,Canada,CA-ON,0.048,238,2022-02-02,5.1,43.655,-79.337,5
1320,dpz9h25,"POLYGON((-79.2622375488281 43.76953125, -79.26...",43.770,-79.262,43.771,-79.261,POINT(-79.26142 43.77018),43.770,-79.261,Toronto,NaN,Ontario,Canada,CA-ON,0.013,15,2022-02-02,5.1,43.770,-79.261,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,dpz88qd,"POLYGON((-79.4393920898438 43.7173461914062, -...",43.717,-79.439,43.719,-79.438,POINT(-79.43835 43.7174),43.717,-79.438,Toronto,NaN,Ontario,Canada,CA-ON,0.009,5,2022-02-02,5.1,43.717,-79.438,1
565,dpz88q6,"POLYGON((-79.4393920898438 43.7159729003906, -...",43.716,-79.439,43.717,-79.438,POINT(-79.43847 43.71725),43.717,-79.438,Toronto,NaN,Ontario,Canada,CA-ON,0.003,3,2022-02-02,5.1,43.717,-79.438,1
564,dpz88q6,"POLYGON((-79.4393920898438 43.7159729003906, -...",43.716,-79.439,43.717,-79.438,POINT(-79.43847 43.71725),43.717,-79.438,Toronto,NaN,Ontario,Canada,CA-ON,0.003,3,2022-02-02,5.1,43.717,-79.438,1
563,dpz8ery,"POLYGON((-79.3020629882812 43.7242126464844, -...",43.724,-79.302,43.726,-79.301,POINT(-79.30095 43.72478),43.725,-79.301,Toronto,NaN,Ontario,Canada,CA-ON,0.014,3,2022-02-02,5.1,43.725,-79.301,1


In [11]:
hazardous_merge.to_csv('Hazardous_KSI_Merge.csv')